# Predicting Check-ins of Foursquare Users in Tokyo

## 4 - Machine Learning

### Import libraries

In [1]:
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

### Load engineered features

In [2]:
df = pd.read_csv('df_afterwrangling.csv',
                 index_col=0,
                 parse_dates=[0],
                 date_parser=pd.to_datetime,
                 infer_datetime_format=True)

In [3]:
df_creatures_of_habit = pd.read_csv('df_creatures_of_habit.csv',
                                    index_col=0,
                                    parse_dates=[0],
                                    date_parser=pd.to_datetime,
                                    infer_datetime_format=True
                                    )
df_irregular_creatures_of_habit = pd.read_csv('df_irregular_creatures_of_habit.csv',
                                              index_col=0,
                                              parse_dates=[0],
                                              date_parser=pd.to_datetime,
                                              infer_datetime_format=True
                                              )

In [4]:
df_creatures_of_habit.tail()

,userid_time_place_0,userid_time_place_1,userid_time_place_2,userid_time_place_3,userid_time_place_4,userid_time_place_5,userid_time_place_6,userid_time_place_7,userid_time_place_8,userid_time_place_9,userid_time_place_10,userid_time_place_11,userid_time_place_12,userid_time_place_13,userid_time_place_14,userid_time_place_15,userid_time_place_16,userid_time_place_17,userid_time_place_18,userid_time_place_19,userid_time_place_20,userid_time_place_21,userid_time_place_22,userid_time_place_23,userid_time_place_24,userid_time_place_25,userid_time_place_26,userid_time_place_27,userid_time_place_28,userid_time_place_29,userid_time_place_30,userid_time_place_31,userid_time_place_32,userid_time_place_33,userid_time_place_34,userid_time_place_35,userid_time_place_36,userid_time_place_37,userid_time_place_38,userid_time_place_39,...,userid_time_place_207,userid_time_place_208,userid_time_place_209,userid_time_place_210,userid_time_place_211,userid_time_place_212,userid_time_place_213,userid_time_place_214,userid_time_place_215,userid_time_place_216,userid_time_place_217,userid_time_place_218,userid_time_place_219,userid_time_place_220,userid_time_place_221,userid_time_place_222,userid_time_place_223,userid_time_place_224,userid_time_place_225,userid_time_place_226,userid_time_place_227,userid_time_place_228,userid_time_place_229,userid_time_place_230,userid_time_place_231,userid_time_place_232,userid_time_place_233,userid_time_place_234,userid_time_place_235,userid_time_place_236,userid_time_place_237,userid_time_place_238,userid_time_place_239,userid_time_place_240,userid_time_place_241,userid_time_place_242,userid_time_place_243,userid_time_place_244,userid_time_place_245,userid_time_place_246
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-02-16 11:34:35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-02-16 11:34:53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-02-16 11:34:55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-02-16 11:35:17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-02-16 11:35:29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_irregular_creatures_of_habit.tail()

,userid_place_0,userid_place_1,userid_place_2,userid_place_3,userid_place_4,userid_place_5,userid_place_6,userid_place_7,userid_place_8,userid_place_9,userid_place_10,userid_place_11,userid_place_12,userid_place_13,userid_place_14,userid_place_15,userid_place_16,userid_place_17,userid_place_18,userid_place_19,userid_place_20,userid_place_21,userid_place_22,userid_place_23,userid_place_24,userid_place_25,userid_place_26,userid_place_27,userid_place_28,userid_place_29,userid_place_30,userid_place_31,userid_place_32,userid_place_33,userid_place_34,userid_place_35,userid_place_36,userid_place_37,userid_place_38,userid_place_39,...,userid_place_207,userid_place_208,userid_place_209,userid_place_210,userid_place_211,userid_place_212,userid_place_213,userid_place_214,userid_place_215,userid_place_216,userid_place_217,userid_place_218,userid_place_219,userid_place_220,userid_place_221,userid_place_222,userid_place_223,userid_place_224,userid_place_225,userid_place_226,userid_place_227,userid_place_228,userid_place_229,userid_place_230,userid_place_231,userid_place_232,userid_place_233,userid_place_234,userid_place_235,userid_place_236,userid_place_237,userid_place_238,userid_place_239,userid_place_240,userid_place_241,userid_place_242,userid_place_243,userid_place_244,userid_place_245,userid_place_246
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-02-16 11:34:35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-02-16 11:34:53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-02-16 11:34:55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-02-16 11:35:17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-02-16 11:35:29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Concatenate all features into one DataFrame

In [6]:
df_ML = pd.concat(
    [df_creatures_of_habit, df_irregular_creatures_of_habit], axis=1)

### Perform Logistic Regression

In [7]:
Xtrain = df_ML['2012-July':'2013-Jan']
Xtest = df_ML['2013-Feb':]
ytrain = df['2012-July':'2013-Jan'].venuecat_encoded
ytest = df['2013-Feb':].venuecat_encoded

clf = LogisticRegression(max_iter=1000)
clf.fit(Xtrain, ytrain)
ytrain_pred = clf.predict(Xtrain)
ytest_pred = clf.predict(Xtest)

### Evaluate model performance

In [8]:
print("Train Accuracy: {0:.2f}%".format(
    accuracy_score(ytrain, ytrain_pred) * 100))
print("Test Accuracy: {0:.2f}%".format(
    accuracy_score(ytest, ytest_pred) * 100))
print("F1 score: {0:.2f}".format(f1_score(ytest, ytest_pred, average='micro')))

Train Accuracy: 87.70%
Test Accuracy: 86.05%
F1 score: 0.86
